In [37]:
import numpy as np
import pandas as pd

In [38]:
df = pd.read_csv( './train_u6lujuX_CVtuZ9i (1).csv' )
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [39]:
df = df.drop('Loan_ID', axis=1)

In [40]:
df.isna().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [41]:
df['Gender'] = df['Gender'].fillna( df['Gender'].mode()[0] )

In [42]:
df['Married'] = df['Married'].fillna( df['Married'].mode()[0] )

## Features
    1. Self-Employed
    2. Applicant Income
    3. Loan Amount
    4. credit history

In [43]:
df['Self_Employed'] = df['Self_Employed'].fillna( df['Self_Employed'].mode()[0] )

In [44]:
df['LoanAmount'] = df['LoanAmount'].fillna( df['LoanAmount'].mean() )

In [47]:
df['Credit_History'] = df['Credit_History'].fillna( df['Credit_History'].mode()[0] )

In [48]:
df[['Self_Employed', 'LoanAmount', 'Credit_History', 'ApplicantIncome', 'Loan_Status']].isna().sum()

Self_Employed      0
LoanAmount         0
Credit_History     0
ApplicantIncome    0
Loan_Status        0
dtype: int64

In [50]:
df['Self_Employed'] = df['Self_Employed'].map({ 'Yes': 1, 'No': 0 })

In [53]:
df['Loan_Status'] = df['Loan_Status'].map( { 'Y': 1, 'N': 0 } )

In [54]:
X = df[['Self_Employed', 'LoanAmount', 'Credit_History', 'ApplicantIncome']]
y = df['Loan_Status']

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42 )

1. Logistic Regression
2. Decision Tree
3. KNN
4. Random Forest
5. Support Vector Machine

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

base_models = [
    ('Logistic Regression', LogisticRegression(max_iter=1000)),
    ('Decision Tree', DecisionTreeClassifier(max_depth=3)),
    ('KNN', KNeighborsClassifier(n_neighbors=5)),
    ('Random Forest', RandomForestClassifier(n_estimators=100)),
    ('SVC', SVC())
]

In [61]:
from sklearn.metrics import accuracy_score

for name, model in base_models:
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    print(f"{name} Accuracy:", accuracy_score(y_test, pred))

Logistic Regression Accuracy: 0.7886178861788617
Decision Tree Accuracy: 0.7886178861788617
KNN Accuracy: 0.6422764227642277
Random Forest Accuracy: 0.7723577235772358
SVC Accuracy: 0.6504065040650406


In [65]:
from xgboost import XGBClassifier

meta_model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3
)


In [66]:
from sklearn.ensemble import StackingClassifier

classifier = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)

classifier.fit( X_train, y_train )

y_pred = classifier.predict( X_test )

In [67]:
print( 'Accuracy: ', accuracy_score( y_test, y_pred ) )

Accuracy:  0.7886178861788617
